In [8]:
import os
import pandas as pd
from datetime import datetime
import openpyxl
from openpyxl.styles import PatternFill

In [9]:
# Define Directories
working_dir = r'C:\Users\ACarrion\OneDrive - F.lli De Cecco di Filippo Fara San Martino S.p.A\Documents\Inventory\WH Inventory Reports\Working_files'
output_dir = r'C:\Users\ACarrion\OneDrive - F.lli De Cecco di Filippo Fara San Martino S.p.A\Documents\Inventory\WH Inventory Reports\Reports'

# Get current date
current_date = datetime.now().strftime("%Y-%m-%d")

# Find the current month's files 
files = [f for f in os.listdir(working_dir) if f.endswith(".xlsx")]

# Read and combine data
all_data = []
for file in files:
    file_path = os.path.join(working_dir,file)
    try:
        df = pd.read_excel(file_path, dtype={"Batch": str})
        df['Source File'] = file
        all_data.append(df[['Source File','Plant', 'Storage Location', 'Material', 'Description', 'Batch', 'Qty_SAP', 'Qty_WH', 'Delta', 'QC Hold Qty PSS', 'Hold Reason ES3',
                            'Manufacture Date', 'Critical Shelf Life Date', 'Expiration date', 'Number of Summers', 'Unit Value', 'Pallet number']])
    except Exception as e:
        print(f"Failed to process {file}: {e}")

In [11]:
# Combine into single DataFrame
if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)

    # Identify first and last quantities per batch
    batch_movement = (
        combined_df.groupby(['Plant', 'Material','Batch'])
        .agg(First_Qty=('Qty_WH', 'first'), 
             Last_Qty=('Qty_WH', 'last'), 
             Expiration_Date=('Expiration date', 'first'),
             Critical_Shelf_Life_Date =('Critical Shelf Life Date', 'first'),
             Description = ('Description', 'first'),
             Location = ('Storage Location', 'last')) # Add last storage location
        .reset_index()
    )

    # Determine if quantity changed
    batch_movement['Batch Movement'] = batch_movement.apply(
        lambda row: 'Changed' if row['First_Qty'] != row['Last_Qty'] else 'Static', axis=1
    )

    # Split into tabs by Plant
    plant_list = ['BP01', 'BP02', 'BP04', 'BP07']
    output_path = os.path.join(output_dir, f"Batch_Movement_Report_{current_date}.xlsx")

    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        for plant in plant_list:
            plant_df = batch_movement[batch_movement['Plant'] == plant].copy()
            if not plant_df.empty:
                plant_df.sort_values(by=['Material', 'Expiration_Date'], ascending=[True, False], inplace=True)
                plant_df.to_excel(writer, sheet_name=f"{plant}", index=False)
            else:
                # Create empty sheet if no data
                pd.DataFrame(columns=['Plant', 'Location', 'Material', 'Description', 'Batch', 'First_Qty', 'Last_Qty', 'Expiration_Date', 'Critical_Shelf_Life_Date', 'Batch Movement']).to_excel(writer, sheet_name=f"{plant}", index=False)
                

    # Highlight changed batches
    workbook = openpyxl.load_workbook(output_path)
    highlight = PatternFill(start_color="FFFF00", end_color='FFFF00', fill_type="solid")

    for sheet in workbook.sheetnames:
        ws = workbook[sheet]
        # Freeze Header row
        ws.freeze_panes = 'A2'
        # Apply Filters to the header row
        ws.auto_filter.ref = ws.dimensions

        for row in ws.iter_rows(min_row=2, max_col=10, values_only=False):
            if row[9].value == 'Changed':
                for cell in row:
                    cell.fill = highlight

    workbook.save(output_path)
    workbook.close()
    print(f"Batch Movement Report generated: {output_path}")
else:
    print("No data to process")
   

Batch Movement Report generated: C:\Users\ACarrion\OneDrive - F.lli De Cecco di Filippo Fara San Martino S.p.A\Documents\Inventory\WH Inventory Reports\Reports\Batch_Movement_Report_2025-03-09.xlsx
